In [1]:
import nltk
import pandas as pd
import sys
import os

from data_loader import DataLoader

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.cluster.util import cosine_distance
from nltk import sent_tokenize, word_tokenize, PorterStemmer

import math

import numpy

import spacy
from spacy.lang.en.stop_words import STOP_WORDS

import networkx as nx

import re

from collections import defaultdict
import string

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
loader = DataLoader(DataLoader.data_path2)
table = loader.load_table()
table.head()

C:\Users\lijia\Desktop\MSAI\Sem A\AI6122\Assignment1\Hearts-of-Iron\data_loader.py:13: FutureWarning: Starting with pandas version 2.0 all arguments of read_json except for the argument 'path_or_buf' will be keyword-only.
  self.table = pd.read_json(data_path, 'records', lines = True);


,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A2VBI34EIGLG9C,B000002NBY,Emily,"[0, 0]","I have given other albums five stars, but ""Gra...",5,Amazing Album--TRUELY Merits 5 Stars,1028332800,"08 3, 2002"
1,A9IZ4H60VZMMT,B0002ZDW80,Herbert West,"[3, 3]",This is the bottom line: If you like New-Wave/...,4,Still cool to listen to,1099267200,"11 1, 2004"
2,A1NVEW628JUS2Y,B0000DD79B,Da Peace Dogg,"[2, 87]","Great, just what we need. A 2-album disc set ...",1,Hippies are gonna love this one,1067472000,"10 30, 2003"
3,A1SJ1GPWF60GU0,B0000DFZZA,"MJ ""MJ""","[9, 9]",Upon hearing ''I'm Like A Bird'' for the very ...,4,No One Trick Pony In My Book,1079827200,"03 21, 2004"
4,A21VBTSS7MSSMO,B000002O6A,"John Marshall Lancaster ""Buffett fan""","[0, 2]","I have known about ""Lizard"" since its release ...",4,Good Disc,1187827200,"08 23, 2007"


In [3]:
#Data Preprocess
#Get stopwords
nltk_stopwords = stopwords.words('english')
nltk_stopwords.append('\n')
spacy_stopwords = list(STOP_WORDS)
merged_sw = nltk_stopwords + list(set(spacy_stopwords) - set(nltk_stopwords))

print("nltk: ", len(nltk_stopwords))
print("spacy: ", len(spacy_stopwords))
print("merge: ", len(merged_sw))
print(merged_sw)

nltk:  180
spacy:  326
merge:  383
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'on

In [4]:
#Create the review text dictionary
reviewText = {}
for i in range(200):
    if table['asin'][i] in reviewText:
        chunk = table['reviewText'][i]
        sentences = nltk.sent_tokenize(chunk)
        for s in sentences:
            reviewText[table['asin'][i]].append(s)
    else:
        reviewText[table['asin'][i]] = []
        chunk = table['reviewText'][i]
        sentences = nltk.sent_tokenize(chunk)
        for s in sentences:
            reviewText[table['asin'][i]].append(s)
print(reviewText)

{'B000002NBY': ['I have given other albums five stars, but "Graceland," quite simply, exceeds all rating.', "After becoming slightly less visible during the late '70s, Paul Simon reappeared in full force with this miraculous, powerful, and timeless album.", 'What is so distinctive about "Graceland" is that it shows a masterful collaboration between Simon and various South African musicians, specifically the outstanding group Ladysmith Black Mambazo.', 'The beauty of this album is that it is NOT an imitation or an exploitation of African musicians, but rather an honest sharing of cultures, a partnership that everyone benefited from.', 'It is a global celebration of the richness of outstanding South African musical expression (Ladysmith B. M. RULES!', 'I JUST SAW THEM IN CONCERT AND--WOW WOW WOW!)', 'in partnership with one of our greatest American musicians.This album is real.', 'It is at once raw and refined, traditional and modern.', 'Rather than emphasizing the suffering and rage of 

In [5]:
def data_preprocess(original_review, stop_words):
    # to lowercase
    review = original_review.lower()
    # remove punctuation
    review = re.sub(r'[^\w\s]', ' ', review).strip()

    lemmatizer = WordNetLemmatizer()
    review_words = review.split(' ')
    review_words_tag = nltk.pos_tag(review_words)
    processed_review = ""
    # remove stop words&lemma
    for word, tag in review_words_tag:
        wntag = tag[0].lower()
        wntag = wntag if wntag in ['a', 'r', 'n', 'v'] else None
        if not wntag:
            lemma_word = word
        else:
            lemma_word = lemmatizer.lemmatize(word, wntag)
        if lemma_word not in stop_words:
            processed_review = processed_review + lemma_word + " "
    processed_review = " ".join(processed_review.split())
    processed_review = processed_review.strip()
    return processed_review



In [6]:
processed_reviewText = {}
for r in reviewText:
    processed_reviewText[r] = []
    for t in reviewText[r]:
        p = data_preprocess(t, merged_sw)
        processed_reviewText[r].append(p)
print(processed_reviewText)

{'B000002NBY': ['album star graceland simply exceeds rating', 'slightly visible late 70s paul simon reappear force miraculous powerful timeless album', 'distinctive graceland masterful collaboration simon south african musician specifically outstanding group ladysmith black mambazo', 'beauty album imitation exploitation african musician honest sharing culture partnership benefit', 'global celebration richness outstanding south african musical expression ladysmith b rule', 'saw concert wow wow wow', 'partnership greatest american musician album real', 'raw refine traditional modern', 'emphasize suffering rage black south african people time simon place convey authentically exuberant celebration rich vivacious musical culture', 'benefit graceland ethno musical boundary open expose', 'album mark turn point paul simon career point break folk rock society expect blossom new level intense brilliant musicality actually album', 'plan review concise fear goal window', 'impossible write highligh

In [7]:
# collect frequency

def collect_frequency(processed_review):
    word_frequency = defaultdict(lambda:0)
    for v in processed_review.values():
        for i in v:
            tokens = nltk.word_tokenize(i)
            for token in tokens:
                word_frequency[token] += 1  
    return word_frequency

frequency = collect_frequency(processed_reviewText)

In [8]:
frequency = sorted(frequency.items(), key=lambda item: item[1],reverse=True)
print(frequency)

[('album', 497), ('song', 382), ('quot', 338), ('like', 199), ('music', 152), ('track', 146), ('good', 131), ('sound', 122), ('love', 120), ('time', 119), ('great', 116), ('cd', 103), ('band', 100), ('best', 93), ('new', 86), ('rock', 80), ('hit', 73), ('come', 70), ('release', 67), ('34', 65), ('listen', 63), ('fan', 63), ('record', 63), ('lyric', 60), ('way', 60), ('know', 56), ('think', 54), ('little', 53), ('beat', 52), ('classic', 50), ('year', 49), ('pop', 47), ('hear', 47), ('guitar', 46), ('play', 45), ('world', 44), ('want', 44), ('favorite', 43), ('vocal', 42), ('voice', 41), ('single', 41), ('thing', 41), ('work', 40), ('buy', 40), ('life', 40), ('disc', 39), ('feel', 38), ('use', 37), ('find', 37), ('tune', 36), ('rap', 36), ('long', 34), ('include', 34), ('version', 34), ('better', 34), ('big', 34), ('man', 33), ('title', 32), ('debut', 32), ('collection', 31), ('funk', 31), ('boy', 30), ('live', 30), ('old', 30), ('solo', 29), ('pretty', 29), ('people', 28), ('right', 28)

In [9]:
sorted_processed_reviewText = sorted(processed_reviewText.items(), key=lambda item: len(item[1]),reverse=True)
print(sorted_processed_reviewText)

[('B00004YKUI', ['erykah', 'studio sophomore actual album good better', 'mature use cleva lyric song', 'love fact track run', 'care track listing match cd', 'happy givin', 'breakdown 1', 'penitentiary philosophy rock track demonstrate erykah maturity music', 'shock fan use erykah jazz soundin song', 'listen hooked2', 'cha know traditional erykah sound track 02', 'sound exceptional song', 'wonder wrong turn 3', 'life beat hot', 'like rap artist borrow sample beat', 'life easy manage survive', 'turnin 4', 'remix', 'love song', 'song hater erykah', 'good word understand', 'talkin h badu badu 5', 'cleva song word describe erykah', 'phrase love song dress cost nothin seven dollar fly 6', 'hey sugah interlude track transition moment album intro track 07 7', 'booty beat amazing', 'love lyric', 'laugh everytime hear', 'sugah peter yo ni think sweeta want 8', 'kiss neck hesi relate', 'use abuse', 'stand gonna use advantage 9', '2000 song sad', 'content strong', 'talk delapadated building missta

In [10]:
#B00004YKUI 63> B000002NBY 48 > B000F0UV3Q 37
print(len(processed_reviewText['B00004YKUI']))

63


In [11]:
def build_similarity_matrix(sentencess, stopwords=None):
    similarity_matrix = numpy.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stopwords)
    return similarity_matrix

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

def generate_summary(sentences, top_n=5):
    summarize_text = []

    sentence_similarity_martix = build_similarity_matrix(sentences, None)

    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence)
    for i in range(top_n):
          summarize_text.append(" ".join(ranked_sentence[i][1]))

    print("Summarize Text: \n", ". ".join(summarize_text))

In [12]:
sentences = processed_reviewText['B00004YKUI']
generate_summary(sentences, 3)

Indexes of top ranked_sentence order are  [(0.018848101858870488, 'think song sayin carry extra package need stop talkin past relationship life'), (0.018810126359829822, 'erykah real talent studio voice'), (0.0186879070729846, 'hey sugah interlude track transition moment album intro track 07 7'), (0.018517308670499485, 'sound like old time record sayin eye green eat lot vegetable'), (0.01822054991424616, 'mature use cleva lyric song'), (0.01820077040568932, 'sugah peter yo ni think sweeta want 8'), (0.018139627885717234, 'penitentiary philosophy rock track demonstrate erykah maturity music'), (0.018076566404190954, 'phrase love song dress cost nothin seven dollar fly 6'), (0.017936414559005413, 'yall know green eye mean mean hate jealousy heart'), (0.017811151454048334, 'cleva song word describe erykah'), (0.01773655221640976, 'moon reflect light sun'), (0.017674296941448786, 'listen song everytime listen cd 12'), (0.01765614027361282, 'cha know traditional erykah sound track 02'), (0.

In [13]:
# Abstractive Summarizer by Model
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch

model_name = 'google/pegasus-xsum'
torch_device = 'cuda'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

text = """"""
for i in reviewText['B00004YKUI']:
    text+=i
print(text)

batch = tokenizer.prepare_seq2seq_batch(text, truncation=True, padding='longest',return_tensors='pt').to(torch_device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)

print(tgt_text)

Erykah has done it once again.Her studio sophomore but actual third album is a good or even better than her last two.She has really matured and has used "cleva" lyrics in her songs.I love that fact that all of her tracks run into her each other.I could care less if the track listing does not match what's on the cd.Be happy with what she has givin' you.Here's a breakdown:1.Penitentiary Philosophy-A rock track that demonstrates Erykah's maturity with her music.At first, this may shock fans that are used to Erykah's jazz soundin' songs.After the first listen, you will be hooked2.Didn't Cha Know-The more traditional Erykah sound is back at track 02.She sounds exceptional on this song.She wonders if she "made a wrong turn back there somewhere."3.My Life-The beat is hot.Seems like many rap artists borrowed samples from her beats.Her life is not easy but she still manages to survive.There's "no turnin' back."4.... & On-The remix of On & On.I love this song.It's really a song for the haters of

C:\Users\lijia\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:3668: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
C:\Users\lijia\anaconda3\lib\site-packages\transformers\generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_

["In our series of letters from African journalists, film-maker and columnist Adaobi Tricia Nwaubani looks at the 10 best songs from Erykah Badu's third album."]


In [14]:
#rake_nltk to extract keyphrases
from rake_nltk import Rake

r = Rake()

r.extract_keywords_from_text(text)

print(r.get_ranked_phrases()[0:10])
#python-rake to extract keyphrases
import RAKE
rake_object = RAKE.Rake(merged_sw)
def sort_tuple(tup):
    tup.sort(key = lambda x:x[1])
    return tup
keywords = sort_tuple(rake_object.run(text))[10:]
print(keywords)

['seems like many rap artists borrowed samples', 'cheeba sac radio edit )-', 'first part sounds like', 'back ." 4 ....', 'message still comes across', 'three hit albums', 'overplayed radio remix', 'could care less', 'wrong turn back', 'may shock fans']
[('2000-', 0), ('10', 0), ('11', 0), ('12', 0), ('13', 0), ('14', 0), ('15', 0), ('1997', 0), ('actual', 1.0), ('album', 1.0), ('good', 1.0), ('better', 1.0), ('matured', 1.0), ('cleva', 1.0), ('lyrics', 1.0), ('songs', 1.0), ('love', 1.0), ('fact', 1.0), ('care', 1.0), ('match', 1.0), ('happy', 1.0), ('givin', 1.0), ('breakdown', 1.0), ('maturity', 1.0), ('music', 1.0), ('hooked2', 1.0), ('wonders', 1.0), ('life-', 1.0), ('hot', 1.0), ('beats', 1.0), ('life', 1.0), ('easy', 1.0), ('manages', 1.0), ('survive', 1.0), ('turnin', 1.0), ('& on-', 1.0), ('haters', 1.0), ('words', 1.0), ('understand', 1.0), ('$h*', 1.0), ('badu', 1.0), ('cleva-', 1.0), ('word', 1.0), ('phrase', 1.0), ('fly', 1.0), ('interlude', 1.0), ('intro', 1.0), ('booty-',

In [15]:
#TF-IDF


def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue

def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    :rtype: int
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average
def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentenceValue and sentenceValue[sentence[:15]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

In [16]:
total_documents = len(reviewText['B00004YKUI'])
# Create the Frequency matrix of the words in each sentence.
freq_matrix = _create_frequency_matrix(reviewText['B00004YKUI'])
#print(freq_matrix)

# Calculate TermFrequency and generate a matrix
tf_matrix = _create_tf_matrix(freq_matrix)
#print(tf_matrix)

# Creating table for documents per words
count_doc_per_words = _create_documents_per_words(freq_matrix)
#print(count_doc_per_words)

# Calculate IDF and generate a matrix
idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
#print(idf_matrix)

# Calculate TF-IDF and generate a matrix
tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)
#print(tf_idf_matrix)

# Important Algorithm: score the sentences
sentence_scores = _score_sentences(tf_idf_matrix)
#print(sentence_scores)

# Find the threshold
threshold = _find_average_score(sentence_scores)
#print(threshold)

# Important Algorithm: Generate the summary
summary = _generate_summary(reviewText['B00004YKUI'], sentence_scores, 1.3 * threshold)
print(summary)

 Here's a breakdown:1. My Life-The beat is hot. ... & On-The remix of On & On. Booty-The beat is just amazing. I love her lyrics. All of them make me laugh everytime I hear them. The content is strong here. She talks about delapadated buildings and that her name will be misstated. Orange Moon-A lovely ballad. You get it, it's a pun! In Love With You-A lovely ballad with Marley. Green Eyes-WOW! She is authentic and not that buggle-gum stuff that others try to sing. The import is worth the extra money!
